In [0]:
#@title ## Setup {display-mode: "form"}
#@markdown This block downloads and compiles all depencencies to run the other blocks.
#@markdown
#@markdown When the runtime resets or after a long period of inactivity, this block needs to be rerun.

def setup():
    print("Setting up filesystem")
    %cd -q /content
    %mkdir /content/uploaded /content/results /usr/local/share/models &> /dev/null
    !rm sample_data -rf
    
    print("Downloading models")
    !git clone -q https://github.com/nagadomi/waifu2x.git waifu2x-lua
    !cp -r \
    /content/waifu2x-lua/models/vgg_7/art \
    /content/waifu2x-lua/models/vgg_7/photo \
    /usr/local/share/models/ &> /dev/null
    !rm -rf /content/waifu2x-lua
    
    print("Downloading waifu2x-converter-cpp source files")
    !git clone -q https://github.com/DeadSix27/waifu2x-converter-cpp.git waifu2x
    %cd -q /content/waifu2x
    %mkdir out
    %cd -q out
    
    print("Building waifu2x-converter-cpp")
    !cmake .. &> /dev/null
    !make -j4 &> /dev/null
    
    print("Installing waifu2x-converter-cpp")
    !make install &> /dev/null
    !ldconfig
          
    print("Cleaning up buildfiles")
    %cd -q /content
    !rm waifu2x -rf

#@markdown ### Options
#@markdown force_install allows rerunning setup, even if everything seems finished.
#@markdown
#@markdown Default: False
force_install = False #@param {type:"boolean"}

!which waifu2x-converter-cpp > /dev/null
if _exit_code != 0 or force_install:
    setup()
    print("Setup finished.")
else:
    print("Setup already finished.")

In [0]:
#@title ## Upload files {display-mode: "form"}
#@markdown After running this block, a file upload box will show. One or multiple images can be submitted.
#@markdown
#@markdown **Warning: this will remove all current uploads and results!**
#@markdown
#@markdown *Tip: use the "Files" tab on the left to see all relevant files in the "uploaded" and "results" folders.*

from google.colab import files

def upload_files_dialog():
    %cd -q /content
    !rm -rf /content/results/* /content/uploaded/* &>/dev/null
    %cd -q /content/uploaded
    try:
        uploaded = files.upload()
    except:   
        print("\033[1mBrowser session expired. Please rerun this block.\033[0m")
    %cd -q /content
    
upload_files_dialog()

In [0]:
#@title ## Upscale / Denoise {display-mode: "form"}

#@markdown This block applies upscaling, denoising or both to all uploaded images.
#@markdown
#@markdown Use the block below to download the result(s).

#@markdown ## Options

#@markdown art = model trained on anime-style art
#@markdown
#@markdown photo = model trained on color photographs
model = "art" #@param ["art", "photo"]

#@markdown noise = Only noise reduction, scale_ratio is ignored
#@markdown
#@markdown scale = Only upscaling, noise_level is ignored
#@markdown
#@markdown noise-scale = Both upscaling and noise reduction are applied
mode = "noise-scale" #@param ["noise", "scale", "noise-scale"]

#@markdown Any positive decimal is accepted. Note that higher values take exponentially longer.
#@markdown
#@markdown Recommended values are x1.6 or x2.0.
scale_ratio = 2.0 #@param {type:"number"}

#@markdown 0 = Some noise reduction
#@markdown 
#@markdown 3 = Heavy noise reduction
noise_level = 1 #@param ["0", "1", "2", "3"] {type:"raw"}

#@markdown Enable to keep previous results. Allows for trying diffrent combinations and downloading them together.
#@markdown
#@markdown Default: False
keep_results = False #@param {type:"boolean"}

in_dir = "/content/uploaded/"
out_dir = "/content/results/"

if not keep_results:
    !rm -rf {out_dir}/*

## Process

!waifu2x-converter-cpp \
    --scale-ratio {scale_ratio} \
    --noise-level {noise_level} \
    -m {mode} \
    --model-dir {"/usr/local/share/models/" + model} \
    -r 1 \
    -i {in_dir} \
    -o {out_dir} \
    -s

In [0]:
#@title Download generated results {display-mode: "form"}

from google.colab import files
from pathlib import Path
from datetime import datetime

def get_timestamp():
    timestamp = !date +%Y-%m-%d_%H-%M-%S
    return timestamp[0]

def zip_result():
    zipfile = "/content/results/Waifu2x_" + get_timestamp() + ".zip"
    !zip -r {zipfile} results/*
    return zipfile

def get_result_image_paths():
    return Path("/content/results").glob("**/*.*")

def get_result_image_count():
    files = list(get_result_image_paths())
    return len(list(files))

def download_result(zip_results):
    if get_result_image_count() > 0:
        if not zip_results == "always" and (zip_results == "never" or get_result_image_count() == 1):
            for image in get_result_image_paths():
                try:
                    files.download(str(image))
                except:
                    print("\033[1mBrowser session expired. Please rerun this block.\033[0m")
        elif zip_results == "always" or get_result_image_count() > 1:
            zipfile = zip_result()
            print("Downloading result")
            try:
                files.download(zipfile)
                !rm -rf {zipfile}
            except:
                !rm -rf {zipfile}
                print("\033[1mBrowser session expired. Please rerun this block.\033[0m")

#@markdown ### Options
#@markdown auto = more than 1 result will get zipped, single files will just get downloaded
#@markdown
#@markdown never = download each result individually
#@markdown
#@markdown always = always zip results
#@markdown
#@markdown Default: auto
zip_results = "auto" #@param ["auto", "never", "always"]

download_result(zip_results)

In [0]:
#@title Delete uploaded and generated files {display-mode: "form"}
#@markdown Useful if you ran blocks in the wrong order.
!rm -rf /content/uploaded/*
!rm -rf /content/results/*